In [ ]:
import serial
import time
import re
import matplotlib.pyplot as plt
from datetime import datetime

# ---------- MAGNET SETTINGS ----------
coil_constant = 0.06478    # T/A  <-- your coil constant
MAX_CURRENT_A = 12.0       # hard safety limit in Amps


# ---------- BASIC MAGNET HELPERS ----------
def send_cmd(ser, cmd, read_bytes=0):
    """
    Send a command (without terminator) to the 4G and optionally read back.
    4G uses carriage return '\r' as the terminator.
    """
    ser.reset_input_buffer()
    ser.write((cmd + "\r").encode("ascii"))
    time.sleep(0.05)
    if read_bytes > 0:
        return ser.read(read_bytes)
    return b""


def read_field_T(ser, coil_const=coil_constant):
    """
    Ask IMAG? and convert reply to Tesla.
    Handles '0.0000A', '0.123kG', '0.001 T', etc.
    """
    ser.reset_input_buffer()
    ser.write(b"IMAG?\r")
    time.sleep(0.05)

    last_line = b""

    for _ in range(10):
        line = ser.readline()
        if not line:
            continue

        last_line = line
        text = line.decode(errors="ignore").strip()
        if not text:
            continue

        # parse numeric value + optional unit stuck on the end
        m = re.match(r"\s*([+-]?\d*\.?\d+(?:[eE][+-]?\d+)?)([a-zA-Z]*)", text)
        if not m:
            continue

        value = float(m.group(1))
        unit = m.group(2).lower()   # 'a', 'kg', 'g', 't', or ''

        if unit.startswith("a"):        # Amps → Tesla
            return value * coil_const
        elif unit.startswith("t"):      # Tesla
            return value
        elif "kg" in unit:             # kiloGauss
            return value * 0.1         # 1 kG = 0.1 T
        elif unit.endswith("g"):       # Gauss
            return value * 1e-4        # 1 G = 1e-4 T
        else:
            # unknown / no unit: assume Tesla
            return value

    raise RuntimeError(f"No numeric IMAG? found, last line: {last_line!r}")


def set_sweep_limits(ser, I_low_A, I_high_A):
    """
    Program sweep lower and upper limits in Amps.
    (UNITS on the 4G must be A.)
    """
    send_cmd(ser, f"LLIM {I_low_A:.4f}")
    send_cmd(ser, f"ULIM {I_high_A:.4f}")


# ---------- LOCK-IN HELPER ----------
def read_lockin_R_X_Y_Theta(lockin):
    """
    Uses your SR865 Python object:
        X, Y, R, Theta = lockin.data.get_channel_values()
    """
    X, Y, R, Theta = lockin.data.get_channel_values()
    return X, Y, R, Theta


# ---------- FIELD TARGET SEQUENCE: 0 -> Bmax -> Bmin -> 0 ----------
def build_B_targets_0_Bmax_Bmin_0(B_max, B_min, dB):
    """
    Build a stepped field sequence (in Tesla):

        +dB   -> +B_max in +dB steps
        +Bmax -> B_min in -dB steps (through 0 into negative)
        B_min -> 0 in +dB steps

    So the first measured point is at +dB, not exactly at 0 T.
    Assumes B_max > 0, B_min < 0, dB > 0.
    """
    B_max = abs(B_max)
    B_min = -abs(B_min)   # ensure negative
    dB = abs(dB)

    B_targets = []

    # 1) +dB -> +B_max
    B = dB
    while B <= B_max + 1e-12:
        B_targets.append(round(B, 6))
        B += dB

    # 2) +B_max -> B_min
    B = B_max - dB
    while B >= B_min - 1e-12:
        B_targets.append(round(B, 6))
        B -= dB

    # 3) B_min -> 0
    B = B_min + dB
    while B <= 0.0 + 1e-12:
        B_targets.append(round(B, 6))
        B += dB

    return B_targets


# ---------- STEPPED LOOP: 0 T -> Bmax -> Bmin -> 0 T ----------
def hysteresis_stepped_0_Bmax_Bmin_0(
    mag_ser,
    lockin,
    B_max=0.10,          # Tesla (positive)
    B_min=-0.10,         # Tesla (negative)
    dB=0.002,            # Tesla (2 mT)
    hold_s=1.0,          # wait 1 s at each step
    tol_T=0.0005,        # tolerance in Tesla
    max_time_per_step_s=15.0,
    coil_const=coil_constant,
    csv_file=None,       # open file handle or None
):
    """
    Stepped loop:

        0 T -> +B_max -> B_min -> 0 T

    with step size dB (e.g. 2 mT).

    At each step:
      - move magnet until |B - B_target| < tol_T  (or timeout)
      - pause sweep
      - take ONE measurement of (B, X, Y, R, Theta)
      - print it
      - write to CSV (if csv_file is not None)
      - wait hold_s seconds (no extra measurements)
    """

    # 1) Magnet in REMOTE, Amps mode
    send_cmd(mag_ser, "REMOTE")
    send_cmd(mag_ser, "UNITS A")
    print("UNITS? ->", send_cmd(mag_ser, "UNITS?", read_bytes=40))

    # Build sequence of target fields
    B_targets = build_B_targets_0_Bmax_Bmin_0(B_max, B_min, dB)
    print(f"Number of field steps: {len(B_targets)}")
    print("First few targets:", B_targets[:10])
    print("Last few targets:", B_targets[-10:])

    # Safety: check max current required
    B_max_abs = max(abs(B_max), abs(B_min))
    I_max_req = B_max_abs / coil_const
    if abs(I_max_req) > MAX_CURRENT_A:
        raise ValueError(
            f"max(|B_max|,|B_min|)={B_max_abs:.4f} T requires "
            f"I_max={I_max_req:.3f} A, exceeds ±{MAX_CURRENT_A:.3f} A limit."
        )

    # Set sweep limits once for full range
    I_min = -B_max_abs / coil_const
    I_max = +B_max_abs / coil_const
    set_sweep_limits(mag_ser, I_low_A=I_min, I_high_A=I_max)

    # Data arrays: ONE point per step
    B_list = []
    R_list = []
    X_list = []
    Y_list = []
    Theta_list = []
    B_target_list = []

    # Start from wherever we are
    B_now = read_field_T(mag_ser, coil_const)
    I_now = B_now / coil_const
    print(f"Starting stepped loop from B_now = {B_now:.6f} T (~{I_now:.4f} A)")

    for idx, B_target in enumerate(B_targets):
        I_target = B_target / coil_const

        if abs(I_target) > MAX_CURRENT_A:
            raise ValueError(
                f"Step {idx}: I_target={I_target:.3f} A exceeds "
                f"±{MAX_CURRENT_A:.3f} A limit."
            )

        print(
            f"\nStep {idx+1}/{len(B_targets)}: "
            f"target B = {B_target:.6f} T  (I_target ~ {I_target:.4f} A)"
        )

        # Decide sweep direction
        if I_target > I_now + 1e-6:
            send_cmd(mag_ser, f"ULIM {I_target:.4f}")
            send_cmd(mag_ser, "SWEEP UP")
        elif I_target < I_now - 1e-6:
            send_cmd(mag_ser, f"LLIM {I_target:.4f}")
            send_cmd(mag_ser, "SWEEP DOWN")
        else:
            send_cmd(mag_ser, "SWEEP PAUSE")

        # Move until we are close to target (or timeout)
        t0 = time.time()
        while True:
            t = time.time() - t0
            B = read_field_T(mag_ser, coil_const)

            print(f"  moving: t={t:5.2f}s  B={B: .6f} T -> target {B_target: .6f} T")

            if abs(B - B_target) <= tol_T:
                print(f"  reached target within tol_T={tol_T:.6f} T")
                break
            if t >= max_time_per_step_s:
                print("  WARNING: timeout moving to target field.")
                break

            time.sleep(0.1)  # small delay while moving

        # Pause sweep at the step
        send_cmd(mag_ser, "SWEEP PAUSE")

        # Take ONE measurement at this step
        B_meas = read_field_T(mag_ser, coil_const)
        X, Y, R, Th = read_lockin_R_X_Y_Theta(lockin)

        # Real-time print
        print(
            f"  MEASURE: B={B_meas: .6f} T  "
            f"X={X: .6e}  Y={Y: .6e}  R={R: .6e}  Th={Th: .3f} deg"
        )

        # Save internally
        B_list.append(B_meas)
        R_list.append(R)
        X_list.append(X)
        Y_list.append(Y)
        Theta_list.append(Th)
        B_target_list.append(B_target)

        # Write to CSV if provided
        if csv_file is not None:
            csv_file.write(
                f"{B_meas:.8e},{B_target:.8e},{X:.8e},{Y:.8e},{R:.8e},{Th:.6f}\n"
            )
            csv_file.flush()

        # Update current estimate from measured B
        B_now = B_meas
        I_now = B_now / coil_const

        # Wait hold_s seconds WITHOUT extra measurements
        print(f"  waiting {hold_s:.2f} s at this step")
        time.sleep(hold_s)

    # Final: bring magnet back near 0 T
    print("\n=== FINAL: Sweeping to ~0 T ===")

    send_cmd(mag_ser, "SWEEP PAUSE")
    send_cmd(mag_ser, "UNITS A")

    # small safety window around 0 A
    send_cmd(mag_ser, "LLIM -0.10")
    send_cmd(mag_ser, "ULIM 0.10")
    send_cmd(mag_ser, "SWEEP ZERO")

    t0 = time.time()
    while True:
        B = read_field_T(mag_ser, coil_const)
        print(f"Returning to zero... B = {B:.6f} T")

        if abs(B) < tol_T:
            break
        if time.time() - t0 > max_time_per_step_s:
            print("Timeout returning to ~0 T.")
            break

        time.sleep(0.5)

    send_cmd(mag_ser, "SWEEP PAUSE")
    print("Finished near 0 T.\n")

    # Plot R vs B (one point per step)
    plt.figure()
    plt.plot(B_list, R_list, "o", markersize=4)
    plt.xlabel("Magnetic Field B (T)")
    plt.ylabel("Lock-in R (V)")
    plt.title(
        f"Stepped loop: 0 → {B_max:.3f} T → {B_min:.3f} T → 0 "
        f"(dB={dB*1e3:.1f} mT)"
    )
    plt.grid(True)
    plt.show()

    return {
        "B": B_list,
        "B_target": B_target_list,
        "R": R_list,
        "X": X_list,
        "Y": Y_list,
        "Theta": Theta_list,
    }


# ---------- MAIN: OPEN MAGNET, RUN LOOP, CLOSE ----------
if __name__ == "__main__":
    # Lock-in connection (you need to set this up for your system)
    # from srsinst.sr865 import SR865
    # lockin = SR865("TCPIP::192.168.xxx.xxx::INSTR")
    # Make sure 'lockin' exists before running:
    try:
        lockin
    except NameError:
        raise RuntimeError("Define the 'lockin' SR865 object before running this script.")

    # Magnet serial connection
    PORT = "COM3"   # change to your actual COM port
    BAUD = 9600
    TIMEOUT = 1.0   # seconds

    mag_ser = serial.Serial(
        port=PORT,
        baudrate=BAUD,
        bytesize=serial.EIGHTBITS,
        parity=serial.PARITY_NONE,
        stopbits=serial.STOPBITS_ONE,
        timeout=TIMEOUT,
    )

    print("Opened magnet on", PORT)
    mag_ser.reset_input_buffer()
    mag_ser.write(b"*IDN?\r")
    time.sleep(0.2)
    print("IDN:", mag_ser.read(200))

    # Make a unique CSV filename based on date & time
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_filename = f"moke_data_{timestamp}.csv"
    csv_file = open(csv_filename, "w")
    csv_file.write("B_meas_T,B_target_T,X_V,Y_V,R_V,Theta_deg\n")

    # Example parameters: 2 mT steps between +0.10 T and -0.10 T
    B_max_loop = 0.10   # Tesla
    B_min_loop = -0.10  # Tesla
    dB_step = 0.002     # Tesla (2 mT)

    data = hysteresis_stepped_0_Bmax_Bmin_0(
        mag_ser=mag_ser,
        lockin=lockin,
        B_max=B_max_loop,
        B_min=B_min_loop,
        dB=dB_step,
        hold_s=1.0,
        tol_T=0.0005,
        max_time_per_step_s=15.0,
        csv_file=csv_file,
    )

    csv_file.close()
    print("CSV saved as:", csv_filename)

    mag_ser.close()
    print("Magnet serial closed.")
